In [1]:
import pandas as pd
from datetime import datetime
from pprint import pprint as pp
import json
from copy import deepcopy
import os
import shutil
import numpy as np
import random
import itertools

In [2]:
from modules.aux_functions import *
from modules.global_vars import *
from modules.reservation_management import get_reservation_code
from modules.table_manager import manage_table, write_table_data, get_table_data

In [3]:
np.random.seed(583)
random.seed(352)

In [4]:
# LOCAL_COMPETENCE_HOTELS = ["Melia Barajas", "Axor Barajas"]
# LOCAL_HOTEL = "Senator Barajas"

LOCAL_COMPETENCE_HOTELS = ["Axor Barajas", "Senator Barajas"]
LOCAL_HOTEL = "Melia Barajas"

#LOCAL_COMPETENCE_HOTELS = ["Melia Barajas", "Senator Barajas"]
#LOCAL_HOTEL = "Axor Barajas"

## Retrieve competence data

In [5]:
def competence_data():
    """Retrieve the competence data

    :returns: Comptence prices and its correspond clients
    :rtype: Dictionary

    """
    comp_data = {}
    for indi, comp_hotel in enumerate(LOCAL_COMPETENCE_HOTELS):
        comp_table = HOTEL_TABLES_PATH + comp_hotel + '/' + file_name
        if os.path.isfile(comp_table):
            with open(comp_table) as fp:
                comp_data[comp_hotel] = json.load(fp)
        else:
            comp_data[comp_hotel] = {}
    return comp_data

In [6]:
def get_comp_prices():
    comp_prices = {}

    # Añadimos los precios de la competencia en INT
    for comp_hotel_name in comp_data:
        comp_prices[comp_hotel_name] = []
        for comp_price in comp_data[comp_hotel_name]:
            comp_prices[comp_hotel_name].append(int(comp_price))
    return comp_prices

## Main function to build new data structure

In [7]:
def generate_new_layout():
    hotel_data_out = {}
    for price, clients in hotel_data.items():
        p = int(price)
        comp_prices_to_use = {}

        comp_price = {}
        for comp_hotel_name in comp_prices:
            comp_prices_to_use[comp_hotel_name] = []

            # Añadimos si se encuentra el mismo precio en la competencia
            if p in comp_prices[comp_hotel_name]:
                comp_prices_to_use[comp_hotel_name].append(p)
            # Añadimos el siguiente precio superior, si lo hay
            try:
                comp_prices_to_use[comp_hotel_name].append(next(x for x in sorted(comp_prices[comp_hotel_name]) 
                                                                if x > p))
            except Exception as e:
                pass
            # Añadimos el precio inferior, si lo hay
            try:
                comp_prices_to_use[comp_hotel_name].append(next(x for x in sorted(comp_prices[comp_hotel_name], 
                                                                                  reverse=True) if x < p))
            except Exception as e:
                pass

        # Generamos todas las posibles permitaciones de precios para obtener los precios de las competencias
        temp_data = []
        for client in comp_prices_to_use:
            temp_data.append(comp_prices_to_use[client])
        comp_list_prices = list(itertools.product(*temp_data))

        # Para cada conjunto de precios de la competencia obtenemos la clave que se va a utilziar y
        # calculamos la proporción que se le va a asignar
        for tuple_prices in comp_list_prices:
            comp_price_key = ','.join([str(x) for x in tuple_prices])
            comp_price[comp_price_key] = (sum([x for x in tuple_prices])) / p

        # Se calcula y se almacena el número de clientes que se quedan en nuestro hotel
        # en función de los precios, se distribuyen de manera proporcional
        hotel_data_out[price] = {}
        mean_prop= sum(comp_price.values())
        for comp_key, comp_prop in comp_price.items():
            hotel_data_out[price][comp_key] = clients * (comp_prop / mean_prop)
            # hotel_data_out[price][comp_key] = int(round(clients * (comp_prop / mean_prop)))

    hotel_data_out[COMPETENCE_ORDER_KEY] = ",".join(LOCAL_COMPETENCE_HOTELS)
    out_file = out_folder + file_name
    if not os.path.exists(FINAL_TABLES_PATH):
        os.mkdir(FINAL_TABLES_PATH)
    if not os.path.exists(out_folder):
        os.mkdir(FINAL_TABLES_PATH + LOCAL_HOTEL)
    write_table_data(hotel_data_out, out_file)

    return hotel_data_out

In [8]:
def generate_comp_tables(our_hotel_data, file_name):
    
    comp_hotels_names = our_hotel_data[COMPETENCE_ORDER_KEY].split(",")
    our_hotel_data.pop(COMPETENCE_ORDER_KEY, None)
    
    hotel_data_out = {}
    ret_data = {}

    # Ejecutamos el proceso para cada hotel competencia
    for indi, comp_hotel in enumerate(comp_hotels_names):
        comp_data_props = {}
        
        # Generamos la estructura de prcios propios/precios competencia
        # a partir de los datos de nuestro hotel
        for our_price, competence in our_hotel_data.items():
            for comp_price in competence.keys():

                selected_comp_price = comp_price.split(",")[indi]
                temp_comp_price = comp_price.split(",")
                temp_comp_price.insert(ALL_HOTELS.index(HOTEL), str(our_price))
                del temp_comp_price[ALL_HOTELS.index(comp_hotel)]
                comp_price_key = ",".join(temp_comp_price)
                
                # En caso de no haberlo visitado antes ese precio, se inicializa
                # También se inicializa el diccionario final
                if not selected_comp_price in comp_data_props.keys():
                    comp_data_props[selected_comp_price] = {}

                # Se calcula la proporción para cada conjunto de precios
                comp_data_props[selected_comp_price][comp_price_key] = (sum([int(x) for x in temp_comp_price])) \
                            / int(selected_comp_price)

        # Calculamos el diccionario final para la competencia, identico al de nuestro hotel
        # Se recorren los precios y sus proporciones y se calcula el número de clientes para
        # cada precio del hotel competencia y de la que sería su competencia (en esta segunda 
        # competencia, entraría nuestro hotel)
        
        hotel_data_out = {}
        for comp_price in comp_data_props:
            temp_comp_clients = comp_data[comp_hotel][comp_price]
            hotel_data_out[comp_price] = {}
            mean_prop = sum(comp_data_props[comp_price].values())
            
            for comp_key, comp_prop in comp_data_props[comp_price].items():
                hotel_data_out[comp_price][comp_key] = temp_comp_clients * (comp_prop / mean_prop)
                # hotel_data_out[comp_price][comp_key] = int(round(temp_comp_clients * (comp_prop / mean_prop)))
        
        temp_comp_hotels_names = comp_hotels_names[:]
        temp_comp_hotels_names.remove(comp_hotel)
        
        # Generate competences key order
        hotel_data_out[COMPETENCE_ORDER_KEY] = ",".join([HOTEL] + temp_comp_hotels_names)
        
        out_folder = FINAL_TABLES_PATH + comp_hotel + '/'
        out_file = out_folder + file_name
        if not os.path.exists(FINAL_TABLES_PATH):
            os.mkdir(FINAL_TABLES_PATH)
        if not os.path.exists(out_folder):
            os.mkdir(out_folder)
        write_table_data(hotel_data_out, out_file)
        
        ret_data[comp_hotel] = deepcopy(hotel_data_out)
        
    our_hotel_data[COMPETENCE_ORDER_KEY] = ",".join(LOCAL_COMPETENCE_HOTELS)
    return ret_data

## Funciones auxiliares

In [9]:
# TODO: en caso de usar esta función, cambiar también en la función de ARA_V
    
def get_outcome_prob(our_price, comp_price, our_clients, comp_data, comp_hotels_names, hotel_name):
    """Calculate the probability that a client comes to our hotel given our price and competence price

    :param our_price: Self price of the room
    :param comp_price: Competence price for the same room
    :param our_clients: Number of clients that have chosen us in the past
    :returns: Probability of choosing our hotel
    :rtype: Double

    """
    opt_our_hotel = True if HOTEL not in comp_hotels_names else False
    
    total_clients = our_clients
    # Search for competence data to calculate the demand
    for indi, comp_hotel in enumerate(comp_hotels_names):
        if comp_data[comp_hotel]:
            selected_comp_price = comp_price.split(",")[indi]
            temp_comp_price = comp_price.split(",")
            temp_comp_price.insert(ALL_HOTELS.index(hotel_name), str(our_price))
            del temp_comp_price[ALL_HOTELS.index(comp_hotel)]

            temp_comp_price = ",".join(temp_comp_price)
            total_clients += comp_data[comp_hotel][selected_comp_price][temp_comp_price]

        # If we dont have competence data, we asume they have the same clients than us
        else:
            print("Esto significa que hay algo mal")
            total_clients += our_clients

    return our_clients / total_clients

In [10]:
def generate_csv_tables(our_data, comp_data, hotel_name):
    columns = ['p', 'p0_1', 'p0_2', 'clients', 'demand', 'outcome_prob']
    df = pd.DataFrame(columns=columns)
    
    comp_hotels_names = our_data[COMPETENCE_ORDER_KEY].split(",")
    our_data.pop(COMPETENCE_ORDER_KEY, None)
    
    cont = 0
    for p, comp_p in our_data.items():
        for p0, clients in comp_p.items():
            try:
                outcome_prob = get_outcome_prob(p, p0, clients, comp_data, comp_hotels_names, hotel_name)
            except Exception as e:
                pp(hotel_name)
                pp(p)
                pp(p0)
                pp(our_data)
                pp(comp_data)
                input()
            demand = (clients /outcome_prob)
            
            p0_1, p0_2 = p0.split(",")

            p, p0_1, p0_2 = int(p), int(p0_1), int(p0_2)
            df.loc[cont] = [p, p0_1, p0_2, clients, demand, outcome_prob]
            cont += 1
    
    out_folder = FINAL_CSV_PATH + "/" + hotel_name + '/'
    csv_file_name = file_name.split(".")[0] + ".csv"
    out_file = out_folder + csv_file_name
    if not os.path.exists(FINAL_CSV_PATH):
        os.mkdir(FINAL_CSV_PATH)
    if not os.path.exists(out_folder):
        os.mkdir(out_folder)
    df.to_csv(out_file, sep=';', encoding='utf-8')
    
    our_data[COMPETENCE_ORDER_KEY] = ",".join(comp_hotels_names)
    return df

## Main code

In [11]:
my_hotel_path = HOTEL_TABLES_PATH + LOCAL_HOTEL + '/'
out_folder = FINAL_TABLES_PATH + LOCAL_HOTEL + '/'
# file_name = '21_double-superior_breakfast_weekend_ls.json'
if os.path.exists(out_folder):
    shutil.rmtree(out_folder)

for file_name in os.listdir(my_hotel_path):

    table_file = my_hotel_path + file_name
    with open(table_file) as fp:
        hotel_data = json.load(fp)

    # mean_price = np.mean([int(x) for x in hotel_data.keys()])
    comp_data = competence_data()
    comp_prices = get_comp_prices()

    our_hotel_data = generate_new_layout()
    glob_comp_data = generate_comp_tables(our_hotel_data, file_name)

    # Generamos nuestro propio CSV y luego el de la competencia
    df = generate_csv_tables(our_hotel_data, glob_comp_data, HOTEL)    
    for comp_hotel_name, temp_hotel_data in glob_comp_data.items():
        local_comp_data = deepcopy(glob_comp_data)
        local_comp_data.pop(comp_hotel_name, None)
        local_comp_data[HOTEL] = our_hotel_data
 
        df = generate_csv_tables(temp_hotel_data, local_comp_data, comp_hotel_name)